In [19]:
import requests
import time
import random
import sqlite3
from bs4 import BeautifulSoup

In [ ]:
import json

# Load the JSON data
with open(r'C:\Users\Aston\Documents\GitHub\web_scraper\Breached\breached.co_archived_links.json', 'r') as file:
    json_data = json.load(file)

# Insert the data into the SQLite table
for row in json_data:
    cursor.execute('INSERT INTO data VALUES (?, ?, ?, ?, ?, ?)', row)

conn.commit()


with open(r'C:\Users\Aston\Documents\GitHub\web_scraper\Breached\breached.vc_archived_page.json', 'r') as file:
    json_data = json.load(file)

# Insert the data into the SQLite table
for row in json_data:
    cursor.execute('INSERT INTO data VALUES (?, ?, ?, ?, ?, ?)', row)

conn.commit()
conn.close()

In [14]:
def create_tables(c):
    cursor = c.cursor()
    # Create a table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS threads
                (url TEXT, title TEXT, author TEXT, timestamp TEXT, replies TEXT, views TEXT, last_post TEXT)''')
    c.commit()
    cursor.execute('''CREATE TABLE IF NOT EXISTS posts (url TEXT, author TEXT, timestamp TEXT, body TEXT)''')
    c.commit()

In [15]:
user_agents = [ 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148', 
    'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36' 
]

In [ ]:
conn = sqlite3.connect('breached.db')
cursor = conn.cursor()
create_tables(conn)

In [20]:
def scrape_thread(url):
    print(f"scraping: {url[0]}")
    archive_url = f"http://web.archive.org/web/{url[0]}"
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent}
    response = requests.get(archive_url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    threads = soup.find_all("tr",class_='inline_row')
    for thread in threads:
        try:
            title = thread.find('span', class_='subject_new').text.strip()
            author = thread.find('span', class_='author').text.strip().replace(',', '')
            timestamp = thread.find('span', class_='thread_start_datetime').text.strip()
            last_post = thread.find('span', class_='lastpost').find_all('a')[1].text.strip()
            if thread.find('td', class_='trow1 forumdisplay_regular') is not None:
                replies = thread.find_all('td', class_='trow1 forumdisplay_regular')[3].text.strip()
                views = thread.find_all('td', class_='trow1 forumdisplay_regular')[4].text.strip()
            else:
                replies = thread.find_all('td', class_='trow2 forumdisplay_regular')[3].text.strip()
                views = thread.find_all('td', class_='trow2 forumdisplay_regular')[4].text.strip()
            print(f"Name of the thread: {title}")
            # Insert the scraped data into the database

            cursor.execute("INSERT INTO threads VALUES (?,?, ?, ?, ?, ?, ?)",(url[1],title, author, timestamp, replies, views, last_post))

            # Commit the changes and close the connection
            conn.commit()

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            continue

In [21]:
def scrape_post(url):
    print(f"scraping: {url[0]}")
    archive_url = f"http://web.archive.org/web/{url[0]}"
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent}
    response = requests.get(archive_url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    posts = soup.find_all("div",class_='post')
    for post in posts:
        try:
            author = post.find("div", class_="post__user-profile").text.strip()
            body = post.find("div", class_="post_body").text.strip()
            post_date = post.find("span", class_="post_date")

            if post_date.span and post_date.span.has_attr("title"):
                # Case: <span class="post_date"><span title="March 23, 2022">Yesterday</span>, 05:05 AM ...
                timestamp = post_date.span["title"]
            else:
                # Case: <span class="post_date">March 16, 2022, 09:12 PM ...
                timestamp = post_date.text.strip()


            cursor.execute("INSERT INTO threads VALUES (?,?, ?, ?, ?, ?, ?)",(url[1], author, timestamp, body))
            conn.commit()
        except:
            continue


In [22]:
url = ["20220324184706/https://breached.co/showthread.php?tid=27&pid=5088","test"]
scrape_post(url)

scraping: 20220324184706/https://breached.co/showthread.php?tid=27&pid=5088


In [24]:
cursor.execute('''select endtimestamp ||"/" || original as url, original  from pages left join threads on threads.url = original  where mimetype = "text/html" and threads.url is NULL''')
urls = cursor.fetchall()
for url in urls:
    # Check if the url string constain the word forum
    if 'forum' in url[0].lower():
        scrape_thread(url)
        time.sleep(1)
    elif 'thread' in url[0].lower():
        scrape_post(url)
        time.sleep(1)
    else:
        continue
conn.close()

scraping: 20220610043623/https://breached.co/Forum-Android
An error occurred: 'NoneType' object has no attribute 'text'
An error occurred: 'NoneType' object has no attribute 'text'
An error occurred: 'NoneType' object has no attribute 'text'
An error occurred: 'NoneType' object has no attribute 'text'
An error occurred: 'NoneType' object has no attribute 'text'
scraping: 20220429153609/https://breached.co/Forum-Anime-Manga?datecut=9999&prefix=0&sortby=starter&order=asc
An error occurred: 'NoneType' object has no attribute 'text'
An error occurred: 'NoneType' object has no attribute 'text'
An error occurred: 'NoneType' object has no attribute 'text'
An error occurred: 'NoneType' object has no attribute 'text'
An error occurred: 'NoneType' object has no attribute 'text'
An error occurred: 'NoneType' object has no attribute 'text'
An error occurred: 'NoneType' object has no attribute 'text'
An error occurred: 'NoneType' object has no attribute 'text'
An error occurred: 'NoneType' object h